In [34]:
import wfdb
import numpy as np

In [35]:
record_path = 'record/s0010_re'  # Path to the WFDB record without file extension

# Read the record and annotation files
record = wfdb.rdrecord(record_path)
# annotations = wfdb.rdann(record_path, 'atr')

# Access the data and metadata
signal = record.p_signal
sampling_frequency = record.fs
# annotation_labels = annotations.symbol



In [36]:
# Print the first 10 samples of the first channel
print(signal)
print(sampling_frequency)

[[-0.2445 -0.229   0.0155 ... -0.0015  0.06   -0.009 ]
 [-0.2425 -0.2335  0.009  ... -0.0015  0.061  -0.01  ]
 [-0.2415 -0.2345  0.007  ... -0.0035  0.0555 -0.0085]
 ...
 [ 0.152   0.2695  0.118  ...  0.079   0.036   0.031 ]
 [ 0.136   0.256   0.1205 ...  0.081   0.042   0.03  ]
 [ 0.135   0.2585  0.1245 ...  0.081   0.049   0.029 ]]
1000


In [37]:
signal.shape

(38400, 15)

In [38]:
data = np.array([[2],[1]])

In [39]:
channels = 6

In [40]:
one_hot = np.zeros((data.size, channels), dtype=float)
one_hot[np.arange(data.size), data.ravel()] = 1


In [41]:
np.arange(data.size)

array([0, 1])

In [42]:
data.ravel()

array([2, 1])

In [43]:
one_hot

array([[0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [44]:
b = np.argmax(one_hot, axis = 1)

In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast

In [2]:
path = '../data/ptb-xl/'

In [7]:
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')["filename_lr"]

In [8]:
len(Y)

21799

In [9]:
Y.head()

ecg_id
1    records100/00000/00001_lr
2    records100/00000/00002_lr
3    records100/00000/00003_lr
4    records100/00000/00004_lr
5    records100/00000/00005_lr
Name: filename_lr, dtype: object

In [23]:
def load_audio(filename):
    # sample rate is not important, it was used in the audio implementation of this function
    signal, meta = wfdb.rdsamp(filename)

    return signal

In [24]:
def one_hot_encode(data, channels=256):
    one_hot = np.zeros((data.size, channels), dtype=float)
    one_hot[np.arange(data.size), data.ravel()] = 1

    return one_hot

In [25]:
def mu_law_encode(audio, quantization_channels=256):
    """
    Quantize waveform amplitudes.
    Reference: https://github.com/vincentherrmann/pytorch-wavenet/blob/master/audio_data.py
    """
    mu = float(quantization_channels - 1)
    quantize_space = np.linspace(-1, 1, quantization_channels)

    quantized = np.sign(audio) * np.log(1 + mu * np.abs(audio)) / np.log(mu + 1)
    quantized = np.digitize(quantized, quantize_space) - 1

    return quantized

In [13]:
raw_audio = load_audio(path + Y.iloc[0])



In [16]:
input = raw_audio[:,0]

In [17]:
input.shape

(1000,)

In [26]:
input

array([-0.119, -0.116, -0.12 , -0.117, -0.103, -0.097, -0.119, -0.096,
       -0.048, -0.037, -0.032, -0.002, -0.017, -0.115, -0.156, -0.153,
       -0.143, -0.109, -0.153, -0.178, -0.152,  0.043,  0.469,  0.371,
       -0.002, -0.078, -0.132, -0.132, -0.14 , -0.138, -0.179, -0.18 ,
       -0.119, -0.158, -0.138, -0.117, -0.133, -0.126, -0.149, -0.156,
       -0.139, -0.124, -0.073, -0.078, -0.123, -0.121, -0.062, -0.023,
       -0.024,  0.022,  0.021,  0.011, -0.009, -0.042, -0.048, -0.066,
       -0.088, -0.143, -0.169, -0.17 , -0.19 , -0.184, -0.165, -0.195,
       -0.188, -0.178, -0.186, -0.163, -0.169, -0.138, -0.159, -0.184,
       -0.143, -0.157, -0.126, -0.131, -0.155, -0.143, -0.17 , -0.146,
       -0.157, -0.175, -0.141, -0.158, -0.174, -0.167, -0.168, -0.165,
       -0.158, -0.153, -0.119, -0.113, -0.118, -0.106, -0.149, -0.148,
       -0.132, -0.122, -0.13 , -0.126, -0.051, -0.072, -0.098, -0.081,
       -0.06 , -0.069, -0.106, -0.119, -0.148, -0.129, -0.126, -0.152,
      

In [27]:
input = mu_law_encode(input, 256)

In [28]:
input.shape

(1000,)

In [29]:
input

array([ 48,  48,  48,  48,  51,  52,  48,  53,  68,  73,  76, 118,  89,
        49,  42,  42,  44,  50,  42,  39,  42, 184, 237, 232, 118,  57,
        45,  45,  44,  44,  39,  39,  48,  41,  44,  48,  45,  47,  43,
        42,  44,  47,  59,  57,  47,  47,  62,  83,  82, 170, 170, 158,
       100,  70,  68,  61,  54,  44,  40,  40,  37,  38,  40,  37,  38,
        39,  38,  41,  40,  44,  41,  38,  44,  42,  47,  46,  42,  44,
        40,  43,  42,  39,  44,  41,  39,  40,  40,  40,  41,  42,  48,
        49,  48,  50,  43,  43,  45,  47,  46,  47,  66,  59,  52,  56,
        63,  60,  50,  48,  43,  46,  47,  42,  46,  50, 219, 240, 223,
        69,  48,  47,  50,  42,  42,  45,  44,  45,  47,  44,  41,  46,
        47,  45,  45,  45,  50,  54,  61,  67,  53,  55,  87, 169, 182,
       195, 195, 186, 158,  70,  62,  58,  54,  51,  49,  46,  47,  47,
        45,  47,  48,  45,  44,  43,  41,  42,  48,  48,  51,  59,  51,
        52,  55,  52,  56,  50,  49,  64,  58,  55,  58,  52,  6

In [32]:

# input = np.pad(input, [[self.receptive_fields, 0]], 'constant')

input = one_hot_encode(input)

In [38]:
list(input[0]).index(1)

48

In [ ]:
sig_name = ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6']
for i in Y["filename_lr"]:
    signal, meta = wfdb.rdsamp(path+i)
    if meta['n_sig']!=12:
        print("dif n_sig")
        
    if meta["sig_name"]!=sig_name:
        print("dif sig_name")
    
    

In [ ]:
signal, meta = a

In [52]:
signal[:,0].tolist()

[-0.2445,
 -0.2425,
 -0.2415,
 -0.241,
 -0.2315,
 -0.226,
 -0.225,
 -0.2345,
 -0.2345,
 -0.2235,
 -0.2205,
 -0.227,
 -0.232,
 -0.2375,
 -0.2445,
 -0.237,
 -0.224,
 -0.2265,
 -0.2265,
 -0.2275,
 -0.2325,
 -0.226,
 -0.2255,
 -0.211,
 -0.2135,
 -0.2235,
 -0.22,
 -0.216,
 -0.2,
 -0.1965,
 -0.1965,
 -0.209,
 -0.2385,
 -0.252,
 -0.2385,
 -0.2265,
 -0.2345,
 -0.2545,
 -0.2475,
 -0.231,
 -0.23,
 -0.22,
 -0.2145,
 -0.2145,
 -0.2275,
 -0.2245,
 -0.2135,
 -0.2005,
 -0.2065,
 -0.228,
 -0.219,
 -0.2155,
 -0.2155,
 -0.2215,
 -0.2315,
 -0.228,
 -0.219,
 -0.221,
 -0.227,
 -0.2265,
 -0.229,
 -0.2315,
 -0.2355,
 -0.2235,
 -0.2135,
 -0.216,
 -0.211,
 -0.2125,
 -0.2185,
 -0.2115,
 -0.2035,
 -0.201,
 -0.2045,
 -0.2305,
 -0.233,
 -0.2175,
 -0.222,
 -0.225,
 -0.2135,
 -0.2085,
 -0.2145,
 -0.2115,
 -0.2085,
 -0.208,
 -0.2105,
 -0.2155,
 -0.2115,
 -0.204,
 -0.193,
 -0.1775,
 -0.2005,
 -0.19,
 -0.162,
 -0.1925,
 -0.1995,
 -0.1945,
 -0.188,
 -0.194,
 -0.2125,
 -0.192,
 -0.167,
 -0.1615,
 -0.168,
 -0.162,
 -0.157

In [49]:
meta

{'fs': 100,
 'sig_len': 1000,
 'n_sig': 12,
 'base_date': None,
 'base_time': None,
 'units': ['mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV'],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'comments': []}

In [ ]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data


sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [53]:
import librosa

In [54]:
def load_audio(filename, sample_rate=16000, trim=True, trim_frame_length=2048):
    audio, _ = librosa.load(filename, sr=sample_rate, mono=True)
    print(audio.shape)
    audio = audio.reshape(-1, 1)

    if trim > 0:
        audio, _ = librosa.effects.trim(audio, frame_length=trim_frame_length)

    return audio

In [56]:
au = load_audio('record/helloworld.wav', sample_rate=1000, trim=True, trim_frame_length=2048)

In [59]:
au.shape

(104285, 1)

In [60]:
def calc_receptive_fields(layer_size, stack_size):
        layers = [2 ** i for i in range(0, layer_size)] * stack_size
        num_receptive_fields = np.sum(layers)

        return int(num_receptive_fields)

In [61]:
calc_receptive_fields(5,10)

310

In [20]:
def calc_receptive_fields(layer_size, stack_size):
        layers = [2 ** i for i in range(0, layer_size)] * stack_size
        print(layers)
        num_receptive_fields = np.sum(layers)

        return int(num_receptive_fields)

In [21]:
calc_receptive_fields(10,5)

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512]


5115

In [56]:
import os

import numpy as np
import wfdb
import torch
import torch.utils.data as data
import pandas as pd

In [63]:
def load_audio(filename, sample_rate=16000, trim=True, trim_frame_length=2048):
    # sample rate is not important, it was used in the audio implementation of this function
    signal, meta = wfdb.rdsamp(filename)

    return signal


def one_hot_encode(data, channels=256):
    one_hot = np.zeros((data.size, channels), dtype=float)
    one_hot[np.arange(data.size), data.ravel()] = 1

    return one_hot


def one_hot_decode(data, axis=1):
    decoded = np.argmax(data, axis=axis)

    return decoded


def mu_law_encode(audio, quantization_channels=256):
    """
    Quantize waveform amplitudes.
    Reference: https://github.com/vincentherrmann/pytorch-wavenet/blob/master/audio_data.py
    """
    mu = float(quantization_channels - 1)
    quantize_space = np.linspace(-1, 1, quantization_channels)

    quantized = np.sign(audio) * np.log(1 + mu * np.abs(audio)) / np.log(mu + 1)
    quantized = np.digitize(quantized, quantize_space) - 1

    return quantized


def mu_law_decode(output, quantization_channels=256):
    """
    Recovers waveform from quantized values.
    Reference: https://github.com/vincentherrmann/pytorch-wavenet/blob/master/audio_data.py
    """
    mu = float(quantization_channels - 1)

    expanded = (output / quantization_channels) * 2. - 1
    waveform = np.sign(expanded) * (
                   np.exp(np.abs(expanded) * np.log(mu + 1)) - 1
               ) / mu

    return waveform


class Dataset(data.Dataset):
    def __init__(self, data_dir, sample_rate=16000, in_channels=256, trim=True):
        super(Dataset, self).__init__()

        self.in_channels = in_channels
        self.sample_rate = sample_rate
        self.trim = trim

        self.root_path = data_dir
        self.filenames = pd.read_csv(data_dir+'ptbxl_database.csv', index_col='ecg_id')["filename_lr"]

    def __getitem__(self, index):
        filepath = os.path.join(self.root_path, self.filenames.iloc[index])

        raw_audio = load_audio(filepath, self.sample_rate, self.trim)

        input = raw_audio[:,0]
        input = np.pad(input, [[20, 0]], 'constant')
        input = mu_law_encode(input, self.in_channels)
        input = one_hot_encode(input, self.in_channels)

        target = raw_audio[:,1]
        
        target = mu_law_encode(target, self.in_channels)
        target = one_hot_encode(target, self.in_channels)
        print(target.shape)
        return input,target

    def __len__(self):
        return len(self.filenames)


    

In [64]:
dataset = Dataset(data_dir = "../data/ptb-xl/")
print(dataset[0][0].shape)

(1000, 256)
(1020, 256)
